# Libraries

In [65]:
import pandas as pd
import numpy as np
import math
import requests

In [169]:
pd.set_option('display.max_columns', None) # display all columns in DF

# Connection to OpenFDA and data manipulation

In [276]:
payload = {'api_key': 'vJTYEEbNaMLaY4e8yZDGKYXwdmDdsPfZB05G87ea', 'limit': '1000'}
r = requests.get('https://api.fda.gov/drug/drugsfda.json?', params=payload)
data = r.json()

Flatting Nested JSON

In [274]:
df = pd.json_normalize(data['results'], max_level = 1)
df = df.drop(columns = ['submissions', 'products'])

Downloading as Excel-file

In [275]:
df.to_excel("output_openfda.xlsx")

In [175]:
df_test = df.explode('openfda.application_number') # can be used to remove list in data

# Connection to Medicaid and data manipulation

Creating a tables with all datasets and the related identifiers

In [99]:
r = requests.get('https://data.medicaid.gov/api/1/metastore/schemas/dataset/items')
data = r.json()

df_metastore = pd.json_normalize(data, max_level = 1)
#df_metastore = df[df['title'].str.contains('State Drug Utilization Data')]

df_metastore.head()

,bureauCode,programCode,accessLevel,keyword,dataQuality,description,distribution,identifier,title,issued,...,theme,landingPage,contactPoint.hasEmail,contactPoint.fn,publisher.name,@type,accrualPeriodicity,publisher.@type,temporal,references
0,[009:00],[009:076],public,"[cms-64, enrollment, expansion population, med...",True,Total Medicaid Enrollees - VIII Group Break Ou...,"[{'title': 'CSV', 'mediaType': 'text/csv', 'do...",6c114b2c-cb83-559b-832f-4d8b06d6c1b9,Medicaid Enrollment - New Adult Group,2017-11-10T01:05:58.000Z,...,[Enrollment],https://www.medicaid.gov/,Medicaid.gov@cms.hhs.gov,Medicaid.gov,data.medicaid.gov,NaN,NaN,NaN,NaN,NaN
1,[009:38],[009:076],public,"[performance rates, core sets, quality measures]",True,Performance rates on frequently reported healt...,"[{'title': 'CSV', 'mediaType': 'text/csv', 'do...",c1028fdf-2e43-5d5e-990b-51ed03428625,2017 Child and Adult Health Care Quality Measures,2018-08-08T21:21:28.000Z,...,[Quality],https://medicaid.gov,no-reply@data.medicaid.gov,Medicaid.gov,data.medicaid.gov,NaN,NaN,NaN,NaN,NaN
2,[009:38],[009:076],public,"[core sets, performance rates, quality measures]",True,Performance rates on frequently reported healt...,"[{'title': 'CSV', 'mediaType': 'text/csv', 'do...",229d6279-e614-5353-9226-f6a6f37d06c3,2018 Child and Adult Health Care Quality Measures,2019-07-22T16:24:55.000Z,...,[Quality],https://medicaid.gov,no-reply@data.medicaid.gov,Chris Vaughn,data.medicaid.gov,NaN,NaN,NaN,NaN,NaN
3,[009:38],[009:076],public,[medicaid],True,"Provides program names, program features, popu...","[{'title': 'CSV', 'mediaType': 'text/csv', 'do...",8f39b637-9bb1-5894-9062-2c4f2ad70fba,2017 Managed Care Programs by State,2019-04-18T16:46:12.000Z,...,[Enrollment],NaN,no-reply@data.medicaid.gov,Chris Vaughn,data.medicaid.gov,NaN,NaN,NaN,NaN,NaN
4,[009:38],[009:000],public,"[core sets, performance rates, quality measures]",True,Performance rates on frequently reported healt...,"[{'title': 'CSV', 'mediaType': 'text/csv', 'do...",e36d89c0-f62e-56d5-bc7e-b0adf89262b8,2019 Child and Adult Health Care Quality Measu...,2020-07-22T15:30:30.000Z,...,[Quality],https://www.medicaid.gov,mohanraj.balasubramanian@cms.hhs.gov,RajB,data.medicaid.gov,NaN,NaN,NaN,NaN,NaN


In [ ]:
itr = next(df_metastore['title'].iterrows())[1]
    print(itr)

In [102]:
for index, row in df_metastore.iterrows(): # iterating through meta data
    title_i = df_metastore['title']
    identifier_i = df_metastore['identifier']
    print(title_i, identifier_i)
    
    
#    df['Dataset'] = str(title_i)
#    df['Identifier'] = str(identifier_i)

ValueError: key of type tuple not found and not a MultiIndex

Looping through datasets and the related splitting API requests

In [ ]:
start_url = 'https://data.medicaid.gov/api/1/datastore/query/'
middle_url = 'a9cfe5e9-d7d8-5b87-a7db-b45a7daf84fc'
end_url = '/0'

for offset_i in range(0, 4179700, 10000):
    
    if offset_i == 0:
        r = requests.get(start_url + middle_url + end_url, timeout = 10)
        data = r.json()
        df_medicaid = pd.json_normalize(data['results'], max_level = 1)
        
    else:
        offset = offset_i
        offset_url = '?offset=' + str(offset)
        r = requests.get(start_url + middle_url + end_url + offset_url, timeout = 10)
        data = r.json()
        temp_df = pd.json_normalize(data['results'], max_level = 1)
        df_medicaid = df_medicaid.append(temp_df, ignore_index = True)

print(len(df_medicaid))

In [ ]:
df_medicaid

In [153]:
# droping empty rows
df_medicaid['units_reimbursed'] = pd.to_numeric(df_medicaid['units_reimbursed'], errors='coerce')
df_medicaid['total_amount_reimbursed'] = pd.to_numeric(df_medicaid['total_amount_reimbursed'], errors='coerce')
df_medicaid.dropna(subset = ['units_reimbursed', 'total_amount_reimbursed'], inplace = True)

# calculating price per unit
df_medicaid['price_per_unit'] = df_medicaid['total_amount_reimbursed'] / df_medicaid['units_reimbursed']
#df_medicaid = df_medicaid.replace([np.inf, -np.inf], np.nan).dropna(subset = ['units_reimbursed'], inplace = True) # infinite values
df_medicaid.head()

,utilization_type,state,ndc,labeler_code,product_code,package_size,year,quarter,suppression_used,product_name,units_reimbursed,number_of_prescriptions,total_amount_reimbursed,medicaid_amount_reimbursed,non_medicaid_amount_reimbursed,price_per_unit
0,FFSU,XX,47335073686,47335,0736,86,2013,3,false,BUPROPION,79076.00,1654,36079.07,35577.65,501.42,0.456258
1,FFSU,XX,00185007201,00185,0072,01,2013,1,false,LOVASTATIN,87236.00,2493,24649.55,22863.95,1785.6,0.282562
2,MCOU,OR,00003029320,00003,0293,20,2013,3,false,KENALOG-40,137.75,128,1681.05,1668.24,12.81,12.203630
3,FFSU,IA,00173044200,00173,0442,00,2013,3,false,ZOFRAN INJ,43.00,20,531.38,531.38,0,12.357674
5,MCOU,WV,00603576321,00603,5763,21,2013,3,false,SPIRONOLAC,1035.00,30,239.71,233.71,6,0.231604


In [154]:
df_medicaid = df_medicaid.groupby(['labeler_code', 'product_code', 'year'])['price_per_unit'].mean()
df_medicaid = df_medicaid.reset_index()

In [155]:
df_medicaid.to_excel('output_mediciad_2013.xlsx', index = False)